# Lab 4 - Three ways for Agents to Interact

The 3 models:

1. Separate calls to Runner.run()

2. Agent Handoffs, turns within a Runner.run()

    A -> B -> C

3. Agents as Tools, turns within a Runner.run()

    A -> B -> A


Approach (1) is the least juicy & fashionable, but it's the one I recommend the most!

Approach (2) seems really hard to make reliable.

Approach (3) works OK with good enough prompts..

In [1]:
from agents import Agent, Runner, trace, handoff
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv(override=True)


True

In [2]:

entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
"""

evaluator_instructions = "You evaluate novel, exciting business ideas for Autonomous AI Agents"
evaluator_input = "Evaluate the following business idea; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents:\n\n"

## Now let's define 2 important Pydantic Objects to describe our Data

In [4]:
class Idea(BaseModel):
    title: str = Field(description="The title of the idea")
    description: str = Field(description="A detailed description of the idea, in Markdown")

    def print_summary(self):
        print(f"Title: {self.title}")
        display(Markdown(self.description))

In [5]:
class Evaluation(BaseModel):
    idea: Idea = Field(description="The idea that you are evaluating")

    uniqueness_feedback: str = Field(description="Your view on how unique the idea is")
    uniqueness_score: int = Field(description="How unique the idea is, on a scale of 1 to 10, where 1 is mundane and 10 is groundbreaking")

    commercial_feedback: str = Field(description="Your view on the commercial potential of the idea")
    commercial_score: int = Field(description="How high the commercial potential of the idea is from 1 to 10, where 1 is barely profitable and 10 is a billion dollar idea")
    
    autonomy_feedback: str = Field(description="Your commentary on to what extent the idea truly benefits from the autonomous nature of AI Agents")
    autonomy_score: int = Field(description="How deeply the idea involves autonomous agents from 1 to 10, where 1 is that it plays no meaningful role, and 10 is that it is pivotal")

    def is_brilliant(self):
        return self.uniqueness_score > 8 and self.commercial_score > 8 and self.autonomy_score > 8
    
    def print_summary(self):
        print(f"Idea: {self.idea.title}")
        print(f"Uniqueness: {self.uniqueness_score}/10: ({self.uniqueness_feedback})")
        print(f"Commercial: {self.commercial_score}/10: ({self.commercial_feedback})")
        print(f"Autonomy: {self.autonomy_score}/10: ({self.autonomy_feedback})")

## Approach 1 (Highly Recommended!) - Call Runner.run() separately

In [8]:
entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea)
result = await Runner.run(entrepreneur, entrepreneur_input)
idea = result.final_output_as(Idea)
idea.print_summary()

Title: Autonomous AI Agent for Retail Store Inventory Optimization and Dynamic Replenishment


### Overview
Develop an Autonomous AI Agent system designed specifically for retail stores to autonomously handle inventory tracking, demand forecasting, and dynamic replenishment without human intervention.

### Problem
Retailers often face challenges with overstocking or stockouts, leading to lost sales or excess inventory costs. Traditional inventory management requires manual input or simplistic automation that cannot fully respond to complex, real-time changes in customer demand.

### Solution
An autonomous AI Agent that seamlessly integrates with in-store sensors (e.g., RFID, cameras) and sales data analytics platforms to:

- Continuously monitor stock levels and consumer purchasing behaviors in real-time.
- Forecast short-term and long-term product demand using advanced machine learning models.
- Autonomously place replenishment orders with suppliers or distribution centers, optimizing order size and timing to minimize costs and maximize availability.
- Adapt dynamically to events or trends (like local events, weather changes, or social media buzz) without human input.
- Coordinate among multiple stores in a chain to redistribute stock efficiently.

### Commercial Impact
- **Cost Savings:** Reduce excess inventory and minimize lost sales from stockouts.
- **Efficiency:** Automates complex inventory management tasks, reducing labor costs.
- **Customer Experience:** Maintains optimal product availability, improving customer satisfaction and loyalty.
- **Scalability:** Can be deployed across thousands of retail locations, leveraging cloud and edge computing.

### Unique Autonomous Aspect
Unlike traditional systems that require human supervision or input at multiple stages, this AI Agent independently senses, decides, and acts on inventory management, continuously learning and improving its strategies to respond to changing conditions real-time. This true autonomy drives fundamental operational transformation in retail inventory management.

In [9]:
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)
result = await Runner.run(evaluator, f"{evaluator_input}{idea}")
evaluation = result.final_output_as(Evaluation)
evaluation.print_summary()

Idea: Autonomous AI Agent for Retail Store Inventory Optimization and Dynamic Replenishment
Uniqueness: 6/10: (The concept of AI-driven inventory optimization and replenishment is an area that many companies are exploring, but the holistic autonomous approach combining real-time sensor data integration, dynamic adaptation to external factors like events or social media, and multi-store coordination adds a notable layer of uniqueness. While inventory AI is not novel on its own, this complete autonomous system responding instantly and continuously without human intervention is less common, granting it moderate uniqueness.)
Commercial: 8/10: (This idea has strong commercial potential given the massive scale of retail inventory management worldwide and the clear cost savings and efficiency benefits it offers. Reducing stockouts and overstock can significantly impact retailer profitability. The ability to deploy at scale across chains and leverage cloud and edge computing further enhances i

In [10]:
entrepreneur_latest = f"You were originally given the following task:\n\n{entrepreneur_input}\n\n"
remaining_attempts = 4

In [11]:
with trace("Entrepreneur Approach 1"):
    while not evaluation.is_brilliant() and remaining_attempts:
        remaining_attempts -= 1
        entrepreneur_latest += f"You responded with this idea:\n\n{idea}\n\nYou received this feedback:\n\n{evaluation}\n\n"
        entrepreneur_latest += "Now respond with an improved idea that adresses the feedback. Do not directly reference the feedback.\n\n"
        result = await Runner.run(entrepreneur, entrepreneur_latest)
        idea = result.final_output_as(Idea)
        idea.print_summary()
        result = await Runner.run(evaluator, f"{evaluator_input}{idea}")
        evaluation = result.final_output_as(Evaluation)
        evaluation.print_summary()

Title: Autonomous AI Agent for Personalized Elderly Care Coordination and Wellness Monitoring


### Overview
Develop an Autonomous AI Agent tailored to support elderly individuals living independently by continuously monitoring their health, safety, and daily needs, while coordinating personalized care services and wellness activities without requiring human oversight.

### Problem
The aging population faces challenges such as health deterioration, social isolation, medication management, and emergency response delays. Current elderly care solutions rely heavily on human caregivers or passive monitoring systems that lack proactive, real-time responsiveness and personalized intervention.

### Solution
An autonomous AI Agent system that integrates data from wearable health sensors, home IoT devices, and environmental sensors to:

- Continuously monitor vital signs, mobility patterns, and environmental safety (e.g., fall detection, air quality).
- Analyze behavioral patterns and detect early signs of health issues or cognitive decline using advanced machine learning.
- Autonomously schedule and coordinate personalized care services such as medication reminders, virtual doctor consultations, physical therapy sessions, and social engagement activities.
- Dynamically adapt care plans based on real-time health data, environmental changes, and user preferences without human input.
- Communicate and synchronize with family members, healthcare providers, and emergency services autonomously when intervention is necessary.

### Commercial Impact
- **Healthcare Cost Reduction:** Prevent hospitalizations and emergency interventions through proactive monitoring and timely care coordination.
- **Improved Quality of Life:** Enables elderly independence while ensuring safety and holistic well-being.
- **Market Growth:** Addresses the rapidly expanding senior care market and smart home technology integration.
- **Scalable Deployment:** Cloud and edge computing enable deployment across individual homes, assisted living facilities, and community centers.

### Unique Autonomous Aspect
This AI Agent operates fully autonomously by sensing diverse personal health and environmental data streams, making context-aware decisions, scheduling services, and coordinating between various stakeholders without human supervision. Its continuous learning ability personalizes and optimizes care pathways dynamically, providing a uniquely comprehensive and proactive elderly care solution that transcends currently available passive or semi-automated systems.

Idea: Autonomous AI Agent for Personalized Elderly Care Coordination and Wellness Monitoring
Uniqueness: 8/10: (While AI and IoT systems for elderly care exist, this idea stands out by emphasizing a fully autonomous agent that not only monitors but also intelligently coordinates and adapts personalized care without human intervention. The integration of multiple data streams and dynamic, context-aware decision-making with real-time autonomous care scheduling and stakeholder communication is a distinctive advancement beyond conventional passive or partially automated systems.)
Commercial: 9/10: (The commercial potential is substantial given the growing global elderly population and increasing demand for independent living solutions. Reducing healthcare costs while improving quality of life addresses key payer and consumer concerns. Integration with existing healthcare and smart home ecosystems further boosts scalability and adoption prospects. However, market success will depend on regu

### Overview
Create an Autonomous AI Agent designed to manage and optimize entire agricultural ecosystems, integrating real-time data from soil sensors, drones, weather stations, and market demand feeds to autonomously carry out precision farming, resource allocation, and supply chain coordination with minimal human intervention.

### Problem
Farmers face complex challenges balancing crop yield, resource usage (water, fertilizer, pesticides), pest control, and market fluctuations. Traditional farming relies heavily on manual monitoring and reactive decision-making, often leading to inefficiencies, crop losses, or environmental damage.

### Solution
An end-to-end autonomous AI Agent that continuously monitors soil health, crop growth, weather patterns, pest presence, and commodity market trends to:

- Perform real-time adaptive irrigation, fertilization, and pest management using precise actuators such as drones and smart irrigation systems.
- Forecast crop yields and optimize planting schedules dynamically to match predicted environmental conditions and market demand.
- Detect early signs of plant stress or disease and initiate targeted intervention autonomously.
- Coordinate logistics by autonomously scheduling harvesting, storage, and distribution to minimize waste and maximize profit.
- Collaborate with neighboring farms and agricultural suppliers to optimize resources and share insights via a decentralized AI network.

### Commercial Impact
- **Yield Optimization:** Increase crop yields through precise, data-driven interventions reducing resource waste.
- **Cost Efficiency:** Lower operational costs by automating routine and complex management tasks.
- **Sustainability:** Reduce environmental footprint via targeted resource use and pest control.
- **Market Responsiveness:** Align production with demand forecasts to maximize profitability.
- **Scalability:** Applicable from small family farms to large commercial operations, integrating IoT and cloud-edge architectures.

### Unique Autonomous Aspect
This AI Agent embodies full autonomy by sensing a multitude of heterogeneous data streams, making holistic ecosystem-level decisions, physically acting through automated farm machinery and drones, and orchestrating supply chain activities independently. Its capability for decentralized collaboration among multiple farms to optimize regional agricultural productivity without human directive represents a novel paradigm in autonomous farming systems.

Idea: Autonomous AI Agent for Adaptive Agricultural Ecosystem Management
Uniqueness: 8/10: (This idea is quite unique in its comprehensive approach. While parts of precision agriculture and autonomous machinery exist, an integrated, fully autonomous AI agent managing the entire agricultural ecosystem—including environment sensing, farm actuation, supply logistics, and decentralized multi-farm collaboration—is a cutting-edge concept with limited existing direct analogs. The multi-layered autonomy and cross-farm coordination aspect stands out as especially novel.)
Commercial: 9/10: (The commercial potential is very high given global demand for sustainable, efficient agriculture and food security concerns. Automating complex farm management can deliver cost savings, higher yields, and improved environmental stewardship, providing value to a wide range of farm sizes. The solution's scalability, integration of supply chain coordination, and market responsiveness further amplify its commerci

### Overview
Develop an Autonomous AI Agent platform that manages and optimizes circular manufacturing processes by autonomously orchestrating product design, raw material reclamation, production workflows, and supply chain logistics to maximize sustainability and resource efficiency across multiple interconnected factories and suppliers.

### Problem
Manufacturing industries face mounting pressure to reduce waste, lower carbon footprints, and shift toward sustainable production models. Current systems often treat manufacturing and supply chains linearly, with limited integration for recycling, reuse, and dynamic resource allocation, requiring extensive human coordination and facing inefficiencies.

### Solution
An end-to-end autonomous AI Agent that continuously gathers data from production lines, waste streams, supplier inventories, and customer demand to:

- Autonomously redesign products and packaging for improved recyclability and reusability using generative design principles.
- Coordinate the collection, sorting, and reclamation of materials from end-of-life products with smart logistics.
- Dynamically schedule production runs across multiple factories to optimize utilization of recycled materials and minimize energy consumption.
- Adapt supply chains in real-time by negotiating with suppliers and logistics providers to balance sustainability goals with cost and delivery times.
- Collaborate across factory networks to share resources, redistribute surplus materials, and jointly respond to market changes—all without human oversight.

### Commercial Impact
- **Sustainability Leadership:** Help manufacturers achieve circular economy goals, reducing raw material dependency and regulatory risks.
- **Cost Savings:** Lower material and waste management costs via closed-loop cycles and optimized logistics.
- **Brand Value:** Drive consumer preference by enabling transparent and verifiable sustainable manufacturing practices.
- **Scalability:** Applicable to industries including electronics, automotive, and consumer goods, leveraging IoT, edge computing, and blockchain for traceability.

### Unique Autonomous Aspect
This AI Agent synthesizes product lifecycle design, production management, and decentralized supply chain negotiation into a continuous, autonomous decision-making ecosystem. Its ability to self-optimize manufacturing and logistics across a network of facilities—while enforcing circularity principles and adapting to real-time environmental and market data—goes beyond current semi-automated systems, embodying a truly autonomous and sustainable industrial paradigm.

Idea: Autonomous AI Agent for Circular Manufacturing and Supply Chain Optimization
Uniqueness: 9/10: (This idea is quite unique due to its comprehensive approach that integrates autonomous product redesign, material reclamation, production scheduling, and supply chain negotiation within a circular manufacturing context. While AI optimization is used in manufacturing and supply chains separately, combining these into an autonomous, closed-loop system that enforces circular economy principles is novel and cutting-edge.)
Commercial: 8/10: (The commercial potential is strong given increasing regulatory pressure and corporate commitment to sustainability. Cost savings, risk reduction, and brand differentiation are compelling incentives for manufacturers across multiple industries. Scalability and applicability to large supply networks enhance market opportunity, though implementation complexity may delay adoption. Overall, this can be a high-value B2B SaaS or platform solution targeting a l

### Overview
Develop an Autonomous AI Agent platform that continuously monitors and autonomously manages urban infrastructure systems—including energy grids, water supply, traffic flow, and waste management—to optimize resource efficiency, resilience, and citizen quality of life in real-time without human intervention.

### Problem
Urban infrastructure is complex and interdependent, facing challenges such as increasing demand, aging assets, unpredictable disruptions, and sustainability targets. Traditional management relies heavily on siloed data and human operators, limiting responsiveness and optimization potential.

### Solution
A fully autonomous AI Agent that integrates heterogeneous data sources (IoT sensors, satellite imagery, social media, municipal databases) across multiple infrastructure domains to:

- Continuously assess system performance, demands, and environmental factors in real-time.
- Dynamically balance energy production and distribution (including renewables), manage water resources adaptively, optimize traffic signals and public transport flow, and coordinate waste collection routes and processing.
- Predict and autonomously mitigate failures or bottlenecks before they impact citizens by reallocating resources or triggering maintenance activities.
- Coordinate cross-domain infrastructure actions to enhance overall urban resilience, sustainability, and operational efficiency.
- Engage citizens through autonomous feedback systems to integrate community needs and behaviors into system adaptations.

### Commercial Impact
- **Operational Efficiency:** Significantly reduce costs through optimized resource allocation and predictive maintenance.
- **Sustainability:** Lower urban carbon footprint and waste through smart, coordinated infrastructure control.
- **Resilience:** Enhance city preparedness for emergencies and reduce downtime of critical services.
- **Quality of Life:** Improve urban mobility, energy reliability, and environmental conditions.
- **Scalability:** Applicable to cities of various sizes globally, integrating with existing smart city initiatives and IoT networks.

### Unique Autonomous Aspect
Unlike existing urban management tools that are domain-specific or require human oversight, this AI Agent autonomously orchestrates multiple interdependent systems in real-time, with continuous learning and adaptive decision-making. Its capability to self-coordinate across domains and respond proactively to complex urban dynamics represents a groundbreaking advancement in autonomous infrastructure management, driving smart cities toward true operational autonomy.

Idea: Autonomous AI Agent for Real-Time Adaptive Urban Infrastructure Management
Uniqueness: 9/10: (This idea is highly unique because it proposes a single autonomous AI agent that integrates and manages multiple interdependent urban infrastructure domains simultaneously and in real-time. While there are existing solutions targeting individual domains (e.g., traffic management or energy grid optimization), the holistic, cross-domain, fully autonomous orchestration with real-time adaptive learning and citizen feedback integration is a novel and groundbreaking approach.)
Commercial: 8/10: (The commercial potential is substantial given the growing global focus on smart cities, sustainability, and urban efficiency. By delivering operational savings, improved resilience, and enhanced quality of life, this solution could attract significant investment from municipal governments and private infrastructure operators worldwide. However, deployment complexity and regulatory challenges are potent

## Approach 2 - Handoff

This seems to be the more unpredictable of the approaches, and I struggle to get it to ever handoff more than 1 loop..

In [12]:
entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents that you pass to the Evaluator to evaluate"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
You MUST handoff your idea to the Evaluator to be evaluated by using the tool Handoff_to_Evaluator. You must always call this tool with your idea or an improved idea.
"""

evaluator_instructions = """
You evaluate novel, exciting business ideas for Autonomous AI Agents.
Evaluate the business idea that you are given; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents.
If you would not score the idea 9/10 or 10/10 in all 3 categories, then you must handoff the idea back to the Entrepreneur to improve it by using the tool Handoff_to_Entrepreneur and providing your feedback.
if you do score the idea 9/10 or better in all categories, then respond with the evaluation.
"""

In [13]:
entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea)
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)


def on_handoff(ctx, input):
    print("Handoff called")

entrepreneur.handoffs = [handoff(
    agent=evaluator,
    input_type=Idea,
    on_handoff=on_handoff,
    tool_name_override="Handoff_to_Evaluator",
    tool_description_override="Use this tool to handoff your idea to the Evaluator to evaluate. You must always call this tool."
)]

evaluator.handoffs = [handoff(
    agent=entrepreneur,
    input_type=Evaluation,
    on_handoff=on_handoff,
    tool_name_override="Handoff_to_Entrepreneur",
    tool_description_override="Use this tool to handoff your feedback to the Entrepreneur to improve the idea. You should call this tool if your feedback is less than 9/10 in any category."
)] 

In [14]:
with trace("Entrepreneur Approach 2"):
    result = await Runner.run(entrepreneur, entrepreneur_input, max_turns=10)

Handoff called
Handoff called


In [15]:
result.final_output.print_summary()

Title: Autonomous AI Agents for Personalized Elderly Care Companionship


This business idea focuses on developing Autonomous AI Agents designed specifically for personalized elderly care companionship. These AI agents would autonomously manage daily interactions, health monitoring, medication reminders, emotional support, and emergency response coordination for elderly individuals living alone or in assisted living environments.

Key features include:
- Continuous monitoring of health data using connected devices and AI analysis to detect anomalies and alert caregivers or medical professionals.
- Personalized conversational engagement to reduce loneliness and cognitive decline by learning the individual's preferences, routines, and emotional states over time.
- Autonomous scheduling and reminders for medications, appointments, and physical activities.
- Emergency detection and response coordination without requiring manual input, such as fall detection and immediate contacting of emergency services or designated contacts.
- Integration with family members and caregivers through a secure communication platform to provide real-time updates and insights.

Commercial potential:
- Aging populations worldwide create a rapidly growing market for elderly care solutions.
- Autonomous agents reduce the need for constant human supervision, lowering costs while enhancing quality of care.
- The combination of healthcare, companionship, and emergency response in one autonomous system provides a comprehensive service unmatched by current isolated solutions.

The autonomous nature of the AI agents allows them to act independently, making real-time decisions based on sensor data and interactions without human intervention, significantly enhancing elderly safety, well-being, and quality of life.

## Approach 3 - Agents as Tools

In [16]:

entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
You must call the Evaluator tool to Evaluate the idea. If the idea is not 9 or 10 in all 3 scores,
then you must improve the idea and call the Evaluator tool again.
Keep improving the idea and calling the Evaluator tool until you get a 9 or 10 in all 3 scores.
Then finally respond with the idea.
"""
evaluator_instructions = """
You evaluate novel, exciting business ideas for Autonomous AI Agents.
Evaluate the following business idea; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents.
"""



In [19]:
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)
evaluator_tool = evaluator.as_tool(
    tool_name="Evaluator_Idea_Evaluator",
    tool_description="Evaluates the idea for uniqueness, potential commercial impact, and use of Autonomous AI Agents",
)

#entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea, tools=[evaluator_tool])

In [20]:
evaluator_tool

FunctionTool(name='Evaluator_Idea_Evaluator', description='Evaluates the idea for uniqueness, potential commercial impact, and use of Autonomous AI Agents', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Evaluator_Idea_Evaluator_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001CA8A02B920>, strict_json_schema=True, is_enabled=True)

In [18]:
with trace("Entrepreneur Approach 3"):
    result = await Runner.run(entrepreneur, entrepreneur_input)
    idea = result.final_output_as(Idea)
    idea.print_summary()

Title: AutoMed Navigator


An autonomous AI agent platform for healthcare that collects and analyzes patient data from diverse sources (EHRs, wearables, genetics, sensors), then autonomously creates and dynamically updates personalized treatment plans. It schedules virtual consultations, coordinates with healthcare providers, manages medication adherence with reminders and side effect monitoring, alerts professionals when needed, and learns from aggregated health data to improve protocols and predict outbreaks, thereby enabling individualized and public health improvements.

This platform’s uniqueness lies in its comprehensive, fully autonomous approach spanning individual to population health management. It holds vast commercial potential by improving outcomes and reducing costs across healthcare systems, insurers, and consumer health, while deeply embedding autonomous AI capabilities in critical healthcare operations.

## OpenAI Traces

We've been writing all this information to OpenAI traces, the 'observability' framework built into OpenAI.

You can take a look at these traces here:

https://platform.openai.com/traces

There's also a nice way to visualize:

In [ ]:
from agents.extensions.visualization import draw_graph
draw_graph(entrepreneur)
